### Imports

In [1]:
import nibabel as nib
import matplotlib.pyplot as plt
import os
import cv2
import vtk
import numpy as np

ModuleNotFoundError: No module named 'nibabel'

### Set up constants

In [ ]:
CT_PATH_FILE = os.path.join("assets", "input.nii.gz")
MAXILLA_PATH_FILE = os.path.join("assets", "maxilla.nii.gz")
CT_SLICES_DIR = os.path.join("output", "slices")
MAXILLA_SLICES_DIR = os.path.join("output", "slices")
BLACK_BOUND = 130
SLICE_FILE = os.path.join("output", "slices", "slice_163.png")
MAXILLA_FILE = os.path.join("output", "slices",  "slice_163_maxilla.png")

: 

### Convert niigz file to set of png slices

In [ ]:
if not os.path.exists(CT_SLICES_DIR):
  os.mkdir(CT_SLICES_DIR)

: 

In [ ]:
def convert(nii_path0, output_dir0, suffix=""):
    nii_path = nii_path0

    nii_img = nib.load(nii_path)

    nii_data = nii_img.get_fdata()

    output_dir = output_dir0
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for i in range(nii_data.shape[2]):
        plt.imshow(nii_data[:, :, i], cmap="gray")
        plt.axis("off")
        plt.savefig(os.path.join(output_dir, f"slice_{i}{suffix}.png"), bbox_inches="tight", pad_inches=0)
        plt.clf()

: 

In [ ]:
# Uncoment comands below, if no convertation was made

# convert(CT_PATH_FILE, CT_SLICES_DIR)
# convert(MAXILLA_PATH_FILE, MAXILLA_SLICES_DIR, "_maxilla")

: 

### Observations

In [ ]:
maxilla = cv2.imread(MAXILLA_FILE, cv2.IMREAD_GRAYSCALE)

plt.imshow(maxilla)

: 

In [ ]:
slice = cv2.imread(SLICE_FILE, cv2.IMREAD_GRAYSCALE)

plt.imshow(slice, )

: 

Let's notice, that nasal septum starts from nasal vomer. Nasal vomer can be seen as a bone (aproximate coordinates are 250, 175). 
So, let's start wave algo from nasal vomer and till the end of the nose.

In [ ]:
# select pixels, that belongs to maxilla

white_pixels = []

for i in range(maxilla.shape[0]):
    for j in range(maxilla.shape[1]):
        if maxilla[i][j] > 0:
            white_pixels.append((i, j))


: 

In [ ]:
# select pixels, that is not air. For this we should define what is to be considered as black color

not_air = []

for i in range(slice.shape[0]):
    for j in range(slice.shape[0]):
        if slice[i][j] > BLACK_BOUND:
            not_air.append((i, j))


: 

#### Select start and finish point for algorithm

In [ ]:
start_point = min(white_pixels, key=lambda x: (abs(x[0] - 175), -x[1]))
left_corner = min(filter(lambda x: x[0] < 175, white_pixels), key=lambda x: x[1])
right_corner = min(filter(lambda x: x[0] > 175, white_pixels), key=lambda x: x[1])
finish_point = min(filter(lambda x: 100 <= x[0] <= 250, not_air), key=lambda x: x[1])

: 

#### Dijkstra algorithm for shortest path problem

In [ ]:
import queue

q = queue.PriorityQueue()

def cost(x0, y0, x1, y1):
    return 1 + 2 ** abs(int(slice[x0][y0]) - int(slice[x1][y1]))


W = slice.shape[0]
H = slice.shape[1]
V = W * H
INF = 10 ** 9

dist = [[INF] * H for _ in range(W)]
prev = [[-1] * H for _ in range(W)]

dist[start_point[0]][start_point[1]] = 0

q.put((0, start_point[0], start_point[1]))

while q.qsize() > 0:
    d, x, y = q.get()
    if d != dist[x][y]:
        continue
    
    if x == finish_point[0] and y == finish_point[1]:
        break

    for dx in range(-1, 2):
        for dy in range(-1, 2):
            nx = dx + x
            ny = dy + y
            if 0 <= nx < W and 0 <= ny < H:
                if slice[nx][ny] > BLACK_BOUND:
                    if dist[nx][ny] > dist[x][y] + cost(x, y, nx, ny):
                        dist[nx][ny] = dist[x][y] + cost(x, y, nx, ny)
                        prev[nx][ny] = (x, y)
                        q.put((dist[nx][ny], nx, ny))
    

: 

Let's notice that it is very convinient that metric should be connected with diffrence between colors. 

$$dist(a, b) = f(abs(c_a -c_b))$$

All functions below were tested and give same result:
1. $$f(x) = 1 + 2 ^ x$$
3. $$f(x) = 1 + c * x$$
3. $$f(x) = x$$

### Answer recovery from dijkstra (coloring nasal septum)

In [ ]:
x, y = finish_point

while x != start_point[0] or y != start_point[1]:
    slice[x][y] = 0
    
    i = x - 1
    while slice[i][y] > BLACK_BOUND and i > left_corner[0]:
        slice[i][y] = 0
        i -= 1
    
    i = x + 1
    while slice[i][y] > BLACK_BOUND and i < right_corner[0]:
        slice[i][y] = 0
        i += 1

    x, y = prev[x][y]

plt.imshow(slice)

: 

### Process range of slices with the same algo as above

In [ ]:
FROM = 154
TO = 175 # exclusive

array = np.zeros(shape=(W, H, TO - FROM))

for layer in range(FROM, TO):
    maxilla_file =  os.path.join("output", "slices",  f"slice_{layer}_maxilla.png")
    maxilla = cv2.imread(maxilla_file, cv2.IMREAD_GRAYSCALE)
    slice_file = os.path.join("output", "slices", f"slice_{layer}.png")
    slice = cv2.imread(slice_file, cv2.IMREAD_GRAYSCALE)
    
    white_pixels = []

    for i in range(maxilla.shape[0]):
        for j in range(maxilla.shape[1]):
            if maxilla[i][j] > 0:
                white_pixels.append((i, j))

    
    not_air = []
    BLACK_BOUND = 130

    for i in range(slice.shape[0]):
        for j in range(slice.shape[0]):
            if slice[i][j] > BLACK_BOUND:
                not_air.append((i, j))

    
    start_point = min(white_pixels, key=lambda x: (abs(x[0] - 175), -x[1]))
    left_corner = min(filter(lambda x: x[0] < 175, white_pixels), key=lambda x: x[1])
    right_corner = min(filter(lambda x: x[0] > 175, white_pixels), key=lambda x: x[1])
    finish_point = min(filter(lambda x: 100 <= x[0] <= 250, not_air), key=lambda x: x[1])
    
    def cost(x0, y0, x1, y1):
        return 1 + 2 **(int(slice[x0][y0]) - int(slice[x1][y1]))

    dist = [[INF] * H for _ in range(W)]
    prev = [[-1] * H for _ in range(W)]

    dist[start_point[0]][start_point[1]] = 0

    q.put((0, start_point[0], start_point[1]))

    while q.qsize() > 0:
        d, x, y = q.get()
        if d != dist[x][y]:
            continue
        
        if x == finish_point[0] and y == finish_point[1]:
            break

        for dx in range(-1, 2):
            for dy in range(-1, 2):
                nx = dx + x
                ny = dy + y
                if 0 <= nx < W and 0 <= ny < H:
                    if slice[nx][ny] > BLACK_BOUND:
                        if dist[nx][ny] > dist[x][y] + cost(x, y, nx, ny):
                            dist[nx][ny] = dist[x][y] + cost(x, y, nx, ny)
                            prev[nx][ny] = (x, y)
                            q.put((dist[nx][ny], nx, ny))
    x, y = finish_point

    while x != start_point[0] or y != start_point[1]:
        array[x][y][layer - FROM] = 1
        slice[x][y] = 0
        
        i = x - 1
        while slice[i][y] > BLACK_BOUND and i > left_corner[0]:
            array[i][y][layer - FROM] = 1
            slice[i][y] = 0
            i -= 1
        
        i = x + 1
        while slice[i][y] > BLACK_BOUND and i < right_corner[0]:
            array[i][y][layer - FROM] = 1
            slice[i][y] = 0
            i += 1

        x, y = prev[x][y]

: 

### Import model of nasal septum to vtk

In [ ]:
dims = array.shape
imageData = vtk.vtkImageData()
imageData.SetDimensions(dims[0], dims[1], dims[2])

scalars = vtk.vtkUnsignedCharArray()
scalars.SetNumberOfComponents(1)
scalars.SetName("scalars")
for k in range(dims[2]):
    for j in range(dims[1]):
        for i in range(dims[0]):
            value = array[i,j,k]
            scalars.InsertNextTuple1(value)
imageData.GetPointData().SetScalars(scalars)

mc = vtk.vtkMarchingCubes()
mc.SetInputData(imageData)
mc.ComputeNormalsOn()
mc.SetValue(0, 0.5)

mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(mc.GetOutputPort())

actor = vtk.vtkActor()
actor.SetMapper(mapper)

renderer = vtk.vtkRenderer()
renderer.AddActor(actor)

renderWindow = vtk.vtkRenderWindow()
renderWindow.AddRenderer(renderer)

writer = vtk.vtkPolyDataWriter()
writer.SetFileName("output.vtk")
writer.SetInputConnection(mc.GetOutputPort())
writer.Write()

: 